<a href="https://colab.research.google.com/github/HemanthK1708/Machine-Learning-Models/blob/main/Copy_of_Model_making_for_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Flower classification with TensorFlow Lite Model Maker with TensorFlow 2.0

Model Maker library simplifies the process of adapting and converting a TensorFlow neural-network model to particular input data when deploying this model for on-device ML applications.

This notebook shows an end-to-end example that utilizes this Model Maker library to illustrate the adaption and conversion of a commonly-used image classification model to classify flowers on a mobile device.

## Prerequisites

To run this example, we first need to install serveral required packages, including Model Maker package that in github [repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [ ]:
!pip install tflite-model-maker

     |████████████████████████████████| 194kB 15.7MB/s 
     |████████████████████████████████| 1.0MB 44.3MB/s 
     |████████████████████████████████| 5.5MB 33.4MB/s 
     |████████████████████████████████| 645kB 56.2MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
     |████████████████████████████████| 849kB 46.7MB/s 
     |████████████████████████████████| 92kB 10.4MB/s 
     |████████████████████████████████| 1.2MB 47.7MB/s 
     |████████████████████████████████| 194kB 38.5MB/s 
     |████████████████████████████████| 102kB 14.1MB/s 
     |████████████████████████████████| 37.6MB 84kB/s 
     |████████████████████████████████| 174kB 50.4MB/s 
     |████████████████████████████████| 706kB 43.8MB/s 
     |████████████████████████████████| 358kB 55.0MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=1778c14526cbf837b1d199bc24649c9c3cd2a38bad4d7d243f30cfc23056365a
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dc

In [ ]:
!wget https://storage.googleapis.com/datasetsbyali/fruitdata.zip

--2021-03-02 08:35:48--  https://storage.googleapis.com/datasetsbyali/fruitdata.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.115.128, 172.253.122.128, 142.250.73.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.115.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7601749 (7.2M) [application/zip]
Saving to: ‘fruitdata.zip’

fruitdata.zip       100%[===================>]   7.25M  --.-KB/s    in 0.03s   

2021-03-02 08:35:48 (224 MB/s) - ‘fruitdata.zip’ saved [7601749/7601749]



In [ ]:
!unzip fruitdata.zip

Archive:  fruitdata.zip
   creating: fruitdata/
   creating: fruitdata/apple/
  inflating: fruitdata/apple/0.jpg   
  inflating: fruitdata/apple/1.jpg   
  inflating: fruitdata/apple/10.jpg  
  inflating: fruitdata/apple/100.jpg  
  inflating: fruitdata/apple/101.jpg  
  inflating: fruitdata/apple/102.jpg  
  inflating: fruitdata/apple/103.jpg  
  inflating: fruitdata/apple/104.jpg  
  inflating: fruitdata/apple/105.jpg  
  inflating: fruitdata/apple/106.jpg  
  inflating: fruitdata/apple/107.jpg  
  inflating: fruitdata/apple/108.jpg  
  inflating: fruitdata/apple/109.jpg  
  inflating: fruitdata/apple/11.jpg  
  inflating: fruitdata/apple/110.jpg  
  inflating: fruitdata/apple/111.jpg  
  inflating: fruitdata/apple/112.jpg  
  inflating: fruitdata/apple/113.jpg  
  inflating: fruitdata/apple/114.jpg  
  inflating: fruitdata/apple/115.jpg  
  inflating: fruitdata/apple/116.jpg  
  inflating: fruitdata/apple/117.jpg  
  inflating: fruitdata/apple/118.jpg  
  inflating: fruitdata/apple/

Import the required packages.

In [ ]:
import numpy as np

import tensorflow as tf

from tensorflow_examples.lite.model_maker.core.data_util.image_dataloader import ImageClassifierDataLoader

from tensorflow_examples.lite.model_maker.core.task import image_classifier

from tensorflow_examples.lite.model_maker.core.task.model_spec import mobilenet_v2_spec

from tensorflow_examples.lite.model_maker.core.task.model_spec import ImageModelSpec

import matplotlib.pyplot as plt

## Simple End-to-End Example

### Get the data path

Let's get some images to play with this simple end-to-end example. Hundreds of images is a good start for Model Maker while more data could achieve better accuracy.

In [ ]:
image_path = "/content/fruitdata"

You could replace `image_path` with your own image folders. As for uploading data to colab, you could find the upload button in the left sidebar shown in the image below with the red rectangle. Just have a try to upload a zip file and unzip it. The root file path is the current path.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_image_classification.png" alt="Upload File" width="800" hspace="100">

If you prefer not to upload your images to the cloud, you could try to run the library locally following the [guide](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) in github.

### Run the example
The example just consists of 4 lines of code as shown below, each of which representing one step of the overall process.


1.   Load input data specific to an on-device ML app. Split it to training data and testing data.

In [ ]:
data = ImageClassifierDataLoader.from_folder(image_path)
train_data, test_data = data.split(0.8)

INFO:tensorflow:Load image with size: 1012, num_label: 5, labels: apple, banana, chickoo, orange, strawberry.


2. Customize the TensorFlow model.

In [ ]:
model = image_classifier.create(train_data)

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              3413024   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 3,419,429
Trainable params: 6,405
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5
25/25 [==============================] - 10s 67ms/step - loss: 1.1343 - accuracy: 0.6377
Epoch 2/5
25/25 [==============================] - 2s 66ms/step - loss: 0.4517 - accuracy: 1.0000
Epoch 3/5
25/25 [==============================] - 2s 66ms/step - loss: 0.4333 - accuracy: 1.0000
Epoch 4/5
25/25 [==============

3. Evaluate the model.

In [ ]:
loss, accuracy = model.evaluate(test_data)

7/7 [==============================] - 1s 71ms/step - loss: 0.4058 - accuracy: 1.0000


4.  Export to TensorFlow Lite model.
You could download it in the left sidebar same as the uploading part for your own use.

In [ ]:
model.export(export_dir='.', with_metadata=False)

INFO:tensorflow:Assets written to: /tmp/tmpbz3rwj20/assets


INFO:tensorflow:Assets written to: /tmp/tmpbz3rwj20/assets


INFO:tensorflow:Saving labels in ./labels.txt.


INFO:tensorflow:Saving labels in ./labels.txt.
